In [1]:
import onnx
import onnxruntime
import transformers
import huggingface_hub

/home/jiang784/.conda/envs/cent7/2020.11-py38/PTM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer
from onnxruntime import InferenceSession

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [3]:
session = InferenceSession("./sample_model.onnx", )

In [4]:
print(session)

In [5]:
import onnx
from collections import Counter

def count_custom_layers(model_path):
    # Load the model
    model = onnx.load(model_path)

    # Counter for layer types
    layer_counter = Counter()

    # Iterate over all nodes in the model
    for node in model.graph.node:
        print(node)
        # Assuming custom layers have a specific pattern or identifiable characteristic
        # Update this condition based on your definition of a custom layer
        if 'Custom' in node.op_type.lower():
            layer_counter[node.op_type] += 1

    return layer_counter

# List of your ONNX model file paths
model_paths = ['./model.onnx']

# Dictionary to hold the frequency of custom layers across all models
overall_custom_layers_frequency = Counter()

In [6]:
# Process each model
for path in model_paths:
    custom_layers_in_model = count_custom_layers(path)
    overall_custom_layers_frequency.update(custom_layers_in_model)

# Print the frequencies
print("Custom Layers Frequency Across All Models:")
for layer, count in overall_custom_layers_frequency.items():
    print(f"{layer}: {count}")

input: "attention_mask"
output: "/bert/Unsqueeze_output_0"
name: "/bert/Unsqueeze"
op_type: "Unsqueeze"
attribute {
  name: "axes"
  ints: 1
  type: INTS
}

input: "/bert/Unsqueeze_output_0"
output: "/bert/Unsqueeze_1_output_0"
name: "/bert/Unsqueeze_1"
op_type: "Unsqueeze"
attribute {
  name: "axes"
  ints: 2
  type: INTS
}

input: "/bert/Unsqueeze_1_output_0"
output: "/bert/Cast_output_0"
name: "/bert/Cast"
op_type: "Cast"
attribute {
  name: "to"
  i: 1
  type: INT
}

output: "/bert/Constant_output_0"
name: "/bert/Constant"
op_type: "Constant"
attribute {
  name: "value"
  t {
    data_type: 1
    raw_data: "\000\000\200?"
  }
  type: TENSOR
}

input: "/bert/Constant_output_0"
input: "/bert/Cast_output_0"
output: "/bert/Sub_output_0"
name: "/bert/Sub"
op_type: "Sub"

output: "/bert/Constant_1_output_0"
name: "/bert/Constant_1"
op_type: "Constant"
attribute {
  name: "value"
  t {
    data_type: 1
    raw_data: "\377\377\177\377"
  }
  type: TENSOR
}

input: "/bert/Sub_output_0"
inpu

In [32]:
import json

def get_model_nodes(model_path):
    # Load the model
    model = onnx.load(model_path)

    # List to store node information
    nodes = []

    # Iterate over all nodes in the model
    for node in model.graph.node:
        # Add node details to the list
        nodes.append({"op_type": node.op_type, "name": node.name, "inputs": node.input, "outputs": node.output})

    return nodes

# List of your ONNX model file paths
model_paths = ['MiniLMv6.onnx']

# List to hold all nodes from all models
all_nodes = []

# Process each model
for path in model_paths:
    model_nodes = get_model_nodes(path)
    all_nodes.extend(model_nodes)

# Save all nodes to a JSON file
with open('all_model_nodes.json', 'w') as json_file:
    json.dump(str(all_nodes), json_file, indent=4)

print("All model nodes saved to all_model_nodes.json")

All model nodes saved to all_model_nodes.json


# Creating custom op example in ONNX

In [4]:
import torch

class CustomOp(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        # Define forward pass
        return input * 2  # Example operation

    @staticmethod
    def backward(ctx, grad_output):
        # Define backward pass if necessary
        return grad_output * 2  # Example operation


In [5]:
class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
    
    def forward(self, x):
        # Use the custom operation
        x = CustomOp.apply(x)
        return x


In [8]:
from torch.onnx import register_custom_op_symbolic

def my_custom_op_symbolic(g, input, **kwargs):
    # Ensure the symbolic name matches the 'domain::name' format
    # Adjust 'com.example::CustomOp' as needed for your domain and operation name
    return g.op("com.example::CustomOp", input, **kwargs)

# Adjust the version number as necessary; here it's set to 1
# The corrected format for the symbolic name is used here
register_custom_op_symbolic('com.example::CustomOp', my_custom_op_symbolic, 1)


OnnxExporterError: Failed to register operator com.example::CustomOp. The symbolic name must match the format domain::name, and should start with a letter and contain only alphanumerical characters

In [7]:
dummy_input = torch.randn(10)
torch.onnx.export(MyModel(), dummy_input, "model_with_custom_op.onnx", opset_version=11)


OnnxExporterError: Module onnx is not installed!